### Notebook setup

In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

### Create Engine/Session

In [2]:
import sqlalchemy as sa
import sqlalchemy.orm
from sqlalchemy.ext.asyncio import create_async_engine, AsyncSession

engine = create_async_engine(
    "cockroachdb+asyncpg://root@cockroach:26257/defaultdb", echo=True
)
engine

<IPython.core.display.Javascript object>

In [3]:
async_session = sa.orm.sessionmaker(engine, class_=AsyncSession)
async_session

sessionmaker(class_='AsyncSession', bind=<sqlalchemy.ext.asyncio.engine.AsyncEngine object at 0x7f5ad25cf9c0>, autoflush=True, autocommit=False, expire_on_commit=True)

<IPython.core.display.Javascript object>

### Drop/create tables

In [4]:
from models import Base, User, Message

async with engine.begin() as conn:
    await conn.run_sync(Base.metadata.drop_all)
    await conn.run_sync(Base.metadata.create_all)

ValueError: unknown type: pg_catalog.json

<IPython.core.display.Javascript object>

### Add data

Nick, Eli, and Paul sing 99 bottles of beer...

In [ ]:
users = [User(name="Nick"), User(name="Eli"), User(name="Paul")]
users

In [ ]:
async with async_session() as session:
    session.add_all(users)
    await session.commit()

In [ ]:
import itertools

cycle = itertools.cycle(users)
messages = []

for i in reversed(range(100)):
    content = f"{i} bottles of beer on the wall..."
    user = next(cycle)
    message = Message(user=user, content=content)
    messages.append(message)

async with async_session() as session:
    session.add_all(messages)
    await session.commit()

### Query for data

#### Count queries

In [ ]:
async with async_session() as session:
    # alternatively:
    # statement = sa.select(sa.func.count()).select_from(User)
    statement = sa.select(sa.func.count(User.id))
    results = await session.scalar(statement)


results

In [ ]:
async with async_session() as session:
    statement = sa.select(sa.func.count(Message.id))
    results = await session.scalar(statement)

results

#### Query for user with messages

In [ ]:
async with async_session() as session:
    statement = sa.select(User).options(sa.orm.joinedload(User.messages))
    results = await session.execute(statement)

results

In [ ]:
users = results.unique().scalars().all()
users

In [ ]:
users[0].name

In [ ]:
users[0].messages[:5]

In [ ]:
for m in users[0].messages[:5]:
    print(m.content)

#### Chained statement example

In [ ]:
async with async_session() as session:
    statement = sa.select(User)
    statement = statement.where(User.name == "Eli")
    statement = statement.options(sa.orm.selectinload(User.messages))
    results = await session.execute(statement)

user = results.scalars().first()
user

In [ ]:
user.name

In [ ]:
for m in user.messages[:5]:
    print(m.content)

### Using classmethod queries

In [ ]:
async with async_session() as session:
    messages = await Message.afrom_user(session, "Eli")

len(messages)

In [ ]:
messages[:5]